In [1]:
# Assembling RoM and PoP for option chains

# STATUS: Incomplete

#***          Start ib_insync (run once)       *****
#___________________________________________________

from ib_insync import *
util.startLoop()
ib=IB().connect('127.0.0.1', 7496, clientId=1) # kavi TWS live
# ib=IB().connect('127.0.0.1', 4001, clientId=3) # kavi IBG live

In [2]:
import pandas as pd
import datetime
from scipy.stats import norm
import numpy as np
import math

# For Black Shcoles 
from math import sqrt, exp, log, erf
from decimal import *
getcontext().prec = 5

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'
today = datetime.datetime.now().date()

#*****            Set up the Limits         ****
#_______________________________________________

# Standard Deviation limits
call_probability = 0.97  # for Calls
put_probability = 0.95   # for Puts
sd_days = 252 # no of days for the standard deviation

call_sd = norm.ppf(1-(1-call_probability)/2)
put_sd = norm.ppf(1-(1-put_probability)/2)

# Read the pickles
df_ohlc = pd.read_pickle(datapath+'df_ohlc.pkl')
df_options = pd.read_pickle(datapath+'df_nse_options.pkl')
df_nse_ib = pd.read_pickle(datapath+ 'df_underlying.pkl')

#*****              Black-Scholes         ******
#_______________________________________________

# Ref: - https://ideone.com/fork/XnikMm - Brian Hyde

def get_bsm(undprice, strike, time, rate, sigma, divrate):
    ''' Gets Black Scholes output
    Args:
        (undprice) : Current Stock Price in float
        (strike)   : Strike Price in float
        (time)     : Days to expiration in float
        (rate)     : Time until expiry in days
        (sigma)    : Standard Deviation of stock's return in float
        (divrate)  : Dividend Rate in float
    Returns:
        (delta, call_price, put_price) as a tuple
    '''
    #statistics
#     sigTsquared = sqrt(Decimal(time)/365)*sigma
    sigTsquared = sqrt(time/365)*sigma
    edivT = exp((-divrate*time)/365)
    ert = exp((-rate*time)/365)
    d1 = (log(undprice*edivT/strike)+(rate+.5*(sigma**2))*time/365)/sigTsquared
    d2 = d1-sigTsquared
    Nd1 = (1+erf(d1/sqrt(2)))/2
    Nd2 = (1+erf(d2/sqrt(2)))/2
    iNd1 = (1+erf(-d1/sqrt(2)))/2
    iNd2 = (1+erf(-d2/sqrt(2)))/2

    #Outputs
    callPrice = round(undprice*edivT*Nd1-strike*ert*Nd2, 2)
    putPrice = round(strike*ert*iNd2-undprice*edivT*iNd1, 2)
    delta = Nd1
    
    return (callPrice, putPrice, delta)

#******      Get the Volatility from Annual Standard Deviations  ****
#____________________________________________________________________

# get the max and min of ohlc for each date
df_ohlc['maxp'] = df_ohlc.loc[:, ['O', 'H', 'L', 'C']].max(1) # Max price for calls
df_ohlc['minp'] = df_ohlc.loc[:, ['O', 'H', 'L', 'C']].min(1) # Min price for puts

# Compute the annual standard deviation for calls and puts
df_cSD = df_ohlc[(df_ohlc.D > df_ohlc.D.max()- \
         datetime.timedelta(days=sd_days))].groupby('ibSymbol') \
        ['ibSymbol', 'maxp'].std(ddof=0).rename(columns={'maxp': 'cASD'})

df_pSD = df_ohlc[(df_ohlc.D > df_ohlc.D.max()- \
         datetime.timedelta(days=sd_days))].groupby('ibSymbol') \
        ['ibSymbol', 'minp'].std(ddof=0).rename(columns={'minp': 'pASD'})

df_aSD = df_ohlc[(df_ohlc.D > df_ohlc.D.max()- \
         datetime.timedelta(days=sd_days))].groupby('ibSymbol') \
        ['ibSymbol', 'C'].std(ddof=0).rename(columns={'C': 'ASD'})

df_mean = df_ohlc[(df_ohlc.D > df_ohlc.D.max()- \
         datetime.timedelta(days=sd_days))].groupby('ibSymbol') \
        ['ibSymbol', 'C'].mean().rename(columns={'C': 'Mean'})

df_SD = pd.concat(objs=[df_aSD, df_cSD, df_pSD, df_mean], axis=1).reset_index()

# Convert Expiry column into datetime
df_options.Expiry = pd.to_datetime(df_options.Expiry)

# get the days to expiry for the options
df_options['DTE'] = (df_options.Expiry - datetime.datetime.now()).dt.days

# Determine put or call - option type
df_options['Type'] = np.where(df_options.Strike > df_options.undPrice, 'c', 'p')

# Compute risk-free rate (r) from 91 day T-bills
rate_url = 'https://rbi.org.in/home.aspx'

li = pd.read_html(rate_url)
li_df = li[4].rename(columns = {0: 'Cat', 1: 'Values'})
li_val = li_df.loc[li_df.Cat == '91 day T-bills', 'Values']
r = float((str(li_val).split('\n')[0].split('%')[0].split(' ')[-1:])[0])/100

# Add ibSymbol, margin, lots and standard deviations to df_options
df = df_options.merge(df_nse_ib, on='nseSymbol').merge(df_SD, on='ibSymbol')

# Add risk-free rate 
df['Rate'] = r

df['Sigma'] = df.ASD/df.Mean     # Sigma is volatility measure
df['Dividend'] = 0   # Dividend to be put in future
df.loc[df.DTE == 0, 'DTE'] = 1    # Corrects date to 1 day for last day of expiry

# Compute Black-Scholes and put it to a dataframe
df_bsm = pd.DataFrame(list(np.vectorize(get_bsm)(df.undPrice, df.Strike, 
                      df.DTE, df.Rate, 
                      df.Sigma, df.Dividend))).T.rename(columns = {0: 'cPrice', 
                                                                    1: 'pPrice', 
                                                                    2: 'Delta'})

# The mega dataframe
df = df.join(df_bsm, how='outer')

df['RoM'] = df.pBidPrice/df.marginpct*df.undPrice*252/df.DTE*100

puts_df = df.loc[(df.Type == 'p') & (df.Delta == 1) & (df.RoM > 0), :]

#****    Logic for last-day expiry puts   ****
#_____________________________________________

put_cols = ['ibSymbol', 'undPrice', 'Strike', 'Expiry', 'DTE', 'pLTP', 'pBidPrice', 
            'Type', 'Mlot', 'Delta', 'RoM']

# Filter out 1-day expiry puts
t_puts = puts_df.loc[puts_df.DTE == 1, put_cols].sort_values(['DTE', 'RoM'], ascending=False)

# Compute price variance percentage
t_puts['price_var'] = (t_puts.pBidPrice - t_puts.pLTP)
t_puts['price_var_pct'] = (t_puts.pBidPrice - t_puts.pLTP)/t_puts.pBidPrice

# t_puts1 = t_puts.sort_values(['price_var_pct'])[t_puts.price_var < 0]  # check only

# Establish target price
# ... Use BidPrice + 0.05 for price variance < -10, else use Last Traded Price
# ... For others use BidPrice + 0.05

t_puts['tgt_price'] = np.where(t_puts.price_var < -10, t_puts.pBidPrice+0.05, t_puts.pLTP)
t_puts.loc[t_puts.tgt_price.isna(), 'tgt_price'] = t_puts.pBidPrice + 0.05

In [ ]:
# Qualify the contracts
c_list = [Option(symbol=symbol, exchange='NSE', 
       lastTradeDateOrContractMonth='20181129', strike=str(strike), right='P') 
for symbol, strike in zip(t_puts.ibSymbol, t_puts.Strike)]

contracts = [c for i in range(0, len(c_list), 50) for c in ib.qualifyContracts(*c_list[i: i+50])]

In [8]:
t_puts['Contracts'] = contracts

In [9]:
t_puts[put_cols + ['tgt_price', 'Contracts']].head()

,ibSymbol,undPrice,Strike,Expiry,DTE,pLTP,pBidPrice,Type,Mlot,Delta,RoM,tgt_price,Contracts
8936,MRF,68678.0,66000.0,2018-11-29,1,1499.00,15.00,p,10,1.0,1.594612e+11,15.05,"Option(conId=332265272, symbol='MRF', lastTrad..."
14117,BANKNIFTY,26443.1,25900.0,2018-11-29,1,17.95,16.55,p,20,1.0,1.158441e+11,17.95,"Option(conId=332264177, symbol='BANKNIFTY', la..."
3785,EICHERMOT,23754.0,22500.0,2018-11-29,1,31.50,26.00,p,25,1.0,9.583510e+10,31.50,"Option(conId=331939266, symbol='EICHERMOT', la..."
14116,BANKNIFTY,26443.1,25800.0,2018-11-29,1,12.00,10.60,p,20,1.0,7.419623e+10,12.00,"Option(conId=332264170, symbol='BANKNIFTY', la..."
3784,EICHERMOT,23754.0,22000.0,2018-11-29,1,21.05,20.00,p,25,1.0,7.371931e+10,21.05,"Option(conId=334911429, symbol='EICHERMOT', la..."


In [ ]:
# ib.qualifyContracts(*contracts)

In [11]:
t_puts['Orders'] = [LimitOrder(action='SELL', totalQuantity=mlot, lmtPrice=price) 
for mlot, price in zip(t_puts.Mlot, t_puts.tgt_price)]

In [17]:
limitTrade = [ib.placeOrder(contract, order) 
              for contract, order in zip(t_puts.Contracts, t_puts.Orders)]

ERROR:ib_insync.wrapper:Error 100, reqId 3: Max rate of messages per second has been exceeded:max=50 rec=63 (1)
ERROR:ib_insync.wrapper:Error 100, reqId 3: Max rate of messages per second has been exceeded:max=50 rec=61 (2)
ERROR:ib_insync.wrapper:Error 201, reqId 1892: Order rejected - reason:Transactions in this instrument are limited to closing-only trades.

ERROR:ib_insync.wrapper:Error 201, reqId 1893: Order rejected - reason:Transactions in this instrument are limited to closing-only trades.

ERROR:ib_insync.wrapper:Error 201, reqId 1998: Order rejected - reason:Transactions in this instrument are limited to closing-only trades.

ERROR:ib_insync.wrapper:Error 201, reqId 2068: Order rejected - reason:Transactions in this instrument are limited to closing-only trades.

ERROR:ib_insync.wrapper:Error 201, reqId 2140: Order rejected - reason:Transactions in this instrument are limited to closing-only trades.

ERROR:ib_insync.wrapper:Error 201, reqId 2139: Order rejected - reason:Trans

ibSymbol                                                       MRF
undPrice                                                     68678
Strike                                                       66000
Expiry                                         2018-11-29 00:00:00
DTE                                                              1
pLTP                                                          1499
pBidPrice                                                       15
Type                                                             p
Mlot                                                            10
Delta                                                            1
RoM                                                    1.59461e+11
price_var                                                    -1484
price_var_pct                                             -98.9333
tgt_price                                                    15.05
Contracts        Option(conId=332265272, symbol='MRF', lastTra